In [723]:
import pandas as pd
import numpy as np
import re

%matplotlib inline

In [724]:
# OBJETIVE
    ### Calculate the average hours per day in 2014 that people spent in primary activities

In [725]:
activities = pd.read_csv('atusact_2014.dat', usecols=[0, 26, 13])

In [726]:
activities = activities.sort_values(by=['TUCASEID'], ascending=True)

In [727]:
ac = dict(zip(range(len(list(activities))), list(activities)))

In [728]:
activities.head()

,TUCASEID,TUACTDUR24,TRCODE
0,20140101140007,540,10101
9,20140101140007,605,10101
7,20140101140007,25,181301
6,20140101140007,90,130116
5,20140101140007,20,181301


# Filtering the activity

In [729]:
activities = activities[(activities.TRCODE >= 30000) & (activities.TRCODE < 40000)]

In [730]:
activities

,TUCASEID,TUACTDUR24,TRCODE
1,20140101140007,30,30101
34,20140101140028,180,30109
59,20140101140168,90,30201
63,20140101140168,40,30101
48,20140101140168,45,30101
86,20140101140610,60,30101
207,20140101140702,20,30101
205,20140101140702,55,30101
199,20140101140702,15,30112
195,20140101140702,30,30101


In [731]:
activities = activities.set_index(['TUCASEID', 'TRCODE'])

In [732]:
activities = activities.reset_index(1)

# SUMARIZING

In [733]:
activities = pd.DataFrame(activities, columns=['TUACTDUR24'])

In [734]:
activities = activities.groupby(level='TUCASEID').sum()

In [735]:
activities = activities.reset_index(0)

In [751]:
activities.head()

,TUCASEID,TUACTDUR24
0,20140101140007,30
1,20140101140028,180
2,20140101140168,175
3,20140101140610,60
4,20140101140702,120


## GETTING THE RESPONDENT DATA SET READY FOR MERGE WEIGHTED VALUES

In [737]:
respondent = pd.read_csv('atusresp_2014.dat', usecols=[0, 105])

In [738]:
ac = dict(zip(range(len(list(respondent))), list(respondent)))

In [739]:
respondent

,TUCASEID,TUFINLWGT
0,20140101140007,4.891316e+06
1,20140101140011,2.314737e+07
2,20140101140028,2.094182e+07
3,20140101140063,2.505899e+06
4,20140101140168,1.592823e+07
5,20140101140537,3.622351e+06
6,20140101140559,3.653338e+06
7,20140101140610,1.158960e+07
8,20140101140614,2.019096e+06
9,20140101140639,5.332005e+06


# MERGIN

In [740]:
act_rep = pd.merge(activities, respondent)

In [741]:
act_rep = pd.DataFrame(act_rep, columns=['TUCASEID','TUFINLWGT', 'TUACTDUR24'])

## CREATING A NEW COLUMN WIHT A NEW VALUE OF THE AVERAGE FORMULA 

In [760]:
act_rep['wtd_time'] = act_rep['TUFINLWGT'] * act_rep['TUACTDUR24']

In [761]:
act_rep.head()

,TUCASEID,TUFINLWGT,TUACTDUR24,wtd_time
0,20140101140007,4.891316e+06,30,1.467395e+08
1,20140101140028,2.094182e+07,180,3.769528e+09
2,20140101140168,1.592823e+07,175,2.787441e+09
3,20140101140610,1.158960e+07,60,6.953763e+08
4,20140101140702,6.323639e+06,120,7.588367e+08


In [755]:
deno = act_rep.TUFINLWGT.sum(axis=0)

In [756]:
deno

23598118464.205288

In [757]:
num = act_rep.wtd_time.sum(axis=0)

In [758]:
num

2509243920340.5835

In [759]:
ave =  num / deno

In [753]:
ave

106.33237239429576